In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import optuna

In [ ]:
# make the result reproducible
keras.utils.set_random_seed(42)

In [ ]:
physical_devices = tf.config.list_physical_devices()
print(physical_devices)

In [ ]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

In [ ]:
from utils.constrains import CLASSES, IMAGE_SIZE

BATCH_SIZE = 200
EPOCHS = 60

In [ ]:
from keras import layers, regularizers, Sequential, Input
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy

def define_model(trial):

  conv_regularizer = regularizers.l2(l=trial.suggest_float("conv_regularizer", 0.0001, 0.001)) # 0.0008
  dense_regularizer = regularizers.l2(l=trial.suggest_float("dense_regularizer", 0.005, 0.05)) # 0.01

  dropout_base =trial.suggest_float("dropout_base", 0, 0.4)
  dropout_increment = trial.suggest_float("dropout_increment", 0, 0.25)
  activation_def = 'tanh' # trial.suggest_categorical("activation_def", ["tanh", "relu"])
  
  model = Sequential(
    [
      Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
      layers.Conv2D(6, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(dropout_base),
      layers.Conv2D(16, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(dropout_base + 1*dropout_increment),
      layers.Conv2D(120, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      
      layers.Flatten(),
      layers.Dropout(dropout_base + 2*dropout_increment),
      layers.Dense(84, activation=activation_def, kernel_regularizer=dense_regularizer),
      layers.Dense(CLASSES, activation='softmax'),
    ]
  )
  
  optimizer = Adam(learning_rate=0.001)

  model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=["accuracy"])
  
  return model

In [ ]:
# Defines training and evaluation.
def train_model(model, X_train, y_train):
  
  history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    shuffle=True,
    verbose=0,
  )
  
  return history

def evaluate_trial(history):
  MEASUREMENT_SPAN = 5
  length = len(history.history['accuracy'])
  
  acc = np.mean(history.history['accuracy'][length-MEASUREMENT_SPAN:])
  val_acc = np.mean(history.history['val_accuracy'][length-MEASUREMENT_SPAN:])

  # Should be minimized
  difference = acc - val_acc
  
  return val_acc, difference

In [ ]:
from utils.load_dataset import load_dataset

DIRPATH = './data'
X_train, X_test, y_train, y_test = load_dataset(DIRPATH)

def objective(trial):  
  model = define_model(trial)
  
  history = train_model(model, X_train, y_train)
  val_acc, difference = evaluate_trial(history)
  return val_acc, difference

In [ ]:
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=30, timeout=300)

print("Number of finished trials: ", len(study.trials))